In [1]:
import tensorflow as tf
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cPickle as pickle
import h5py
import random
import scipy.io as sio
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import sys

In [1]:
import tensorflow as tf
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cPickle as pickle
import h5py
import random
import scipy.io as sio
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import sys

class MedTrans(object):
    def init_weights(self, input_dim, output_dim, name=None, std=1.0):
        return tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=std / math.sqrt(input_dim)), name=name)

    def init_bias(self, output_dim, name=None):
        return tf.Variable(tf.zeros([output_dim]), name=name)

    def __init__(self, num_heads, attention_blocks_num, f_dim, visit_num, visit_length, one_hot_input_dim,
                 input_dim, info_dim, voutput_dim, output_dim, vhidden_dim):
        self.num_heads = num_heads
        self.attention_blocks_num = attention_blocks_num
        self.f_dim = f_dim

        self.visit_num = visit_num
        self.visit_length = visit_length

        self.one_hot_input_dim = one_hot_input_dim
        self.input_dim = input_dim
        self.info_dim = info_dim
        self.voutput_dim = voutput_dim
        self.vhidden_dim = vhidden_dim

        self.output_dim = output_dim
        final_output_dim = 2
        self.final_output_dim = 2

        # one-hot后的矩阵参数
        self.Wi = self.init_weights(one_hot_input_dim, input_dim, name='OneHot_w')
        self.bi = self.init_bias(input_dim, name='OneHot_w')

        # visit层的参数
        # 输入是上一层的输出加上病人信息
        self.Wz_v = self.init_weights(input_dim + info_dim, vhidden_dim, name='Update_wx_v')
        self.Uz_v = self.init_weights(vhidden_dim, vhidden_dim, name='Update_wh_v')
        self.bz_v = self.init_bias(vhidden_dim, name='Update_bias_dec')

        self.Wr_v = self.init_weights(input_dim + info_dim, vhidden_dim, name='Reset_wx_v')
        self.Ur_v = self.init_weights(vhidden_dim, vhidden_dim, name='Reset_wh_v')
        self.br_v = self.init_bias(vhidden_dim, name='Reset_bias_v')
        self.Wd_v = tf.ones([1, vhidden_dim], dtype=tf.float32, name='Decay_w_v')
        self.Wh_v = self.init_weights(input_dim + info_dim, vhidden_dim, name='Canditateh_wx_v')
        self.Uh_v = self.init_weights(vhidden_dim, vhidden_dim, name='Canditateh_wh_v')
        self.bh_v = self.init_bias(vhidden_dim, name='Canditateh_bias_v')

        # 输出层
        # visit
        self.Wo = self.init_weights(vhidden_dim, output_dim, name='Output_w')
        self.bo = self.init_bias(output_dim, name='Output_bias')
        # 最终输出全连接
        self.Wf = self.init_weights(output_dim, final_output_dim, name='Final_output_w')
        self.bf = self.init_bias(final_output_dim, name='Final_output_bias')

        # 输入占位符
        # [batch size x seq length x input dim]
        self.inputindex = tf.placeholder(dtype=tf.int32, shape=[None, None])
        self.one_hot_input = tf.one_hot(indices=self.inputindex, depth=one_hot_input_dim, axis=2)  # 输入
        self.labels = tf.placeholder('float', shape=[None, final_output_dim])  # 标签
        self.time = tf.placeholder('float', [None, None])
        self.keep_prob = tf.placeholder(tf.float32)
        self.info = tf.placeholder('float', shape=[None, None, info_dim])  # 患者人口信息



    # 时间衰减函数
    def map_elapse_time(self, t, dim):
        c1 = tf.constant(1, dtype=tf.float32)
        c2 = tf.constant(2.7183, dtype=tf.float32)

        T = tf.div(c1, tf.log(t + c2), name='Log_elapse_time')
        # T = tf.div(c1, tf.add(t , c1), name='Log_elapse_time')
        return T

        # layer normalization

    def ln(self, inputs, epsilon=1e-8, scope="ln"):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            # tf.nn.moments计算均值和方差，[-1]是最后一个轴，也就是参数维度
            mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
            normalized = (inputs - mean) / ((variance + epsilon) ** (.5))

            inputs_shape = inputs.get_shape()
            params_shape = inputs_shape[-1:]
            beta = tf.get_variable("beta", params_shape, initializer=tf.zeros_initializer())
            gamma = tf.get_variable("gamma", params_shape, initializer=tf.ones_initializer())
            outputs = gamma * normalized + beta
        return outputs


    # 对输入的one-hot使用矩阵处理，code的嵌入向量1
    def get_input(self, one_hot_input):
        input = tf.matmul(one_hot_input, self.Wi) + self.bi
        return input

    # 输出层，对h进行全连接输出
    def get_output(self, h):
        output = tf.nn.relu(tf.matmul(h, self.Wo) + self.bo)
        return output

    # 输出层，对h进行全连接输出
    def get_final_output(self, h):
        output = tf.nn.relu(tf.matmul(h, self.Wf) + self.bf)
        return output

    def scaled_dot_product_attention(self, Q, K, V, keep_prob =1.0,scope="scaled_dot_product_attention"):

        # Q: [N, T_q, d_k].
        # K: [N, T_k, d_k].
        # V: [N, T_k, d_v].

        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            d_k = Q.get_shape().as_list()[-1]

            # dot product
            outputs = tf.matmul(Q, tf.transpose(K, [0, 2, 1]))  # (N, T_q, T_k)

            # scale
            outputs /= d_k ** 0.5

            # softmax
            outputs = tf.nn.softmax(outputs)

            attention = tf.transpose(outputs, [0, 2, 1])
            # 可视化attention
            tf.summary.image("attention", tf.expand_dims(attention[:1], -1))

            # dropout
            outputs = tf.nn.dropout(outputs, rate=keep_prob)

            # weighted sum (context vectors)
            outputs = tf.matmul(outputs, V)  # (N, T_q, d_v)

        return outputs


    # multihead_attention
    def multihead_attention(self, queries, keys, values,num_heads=2,keep_prob=1.0,scope="multihead_attention"):

        d_model = queries.get_shape().as_list()[-1]
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            # Linear projections
            Q = tf.layers.dense(queries, d_model,activation=tf.sigmoid, use_bias=True)  # (N, T_q, d_model)
            K = tf.layers.dense(keys, d_model,activation=tf.sigmoid, use_bias=True)  # (N, T_k, d_model)
            V = tf.layers.dense(values, d_model, activation=tf.sigmoid,use_bias=True)  # (N, T_k, d_model)

            # Split and concat
            Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0)  # (h*N, T_q, d_model/h)
            K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0)  # (h*N, T_k, d_model/h)
            V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0)  # (h*N, T_k, d_model/h)

            # Attention
            outputs = self.scaled_dot_product_attention(Q_, K_, V_, keep_prob)

            # Restore shape
            outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2)  # (N, T_q, d_model)

            # Residual connection
            outputs += queries

            # Normalize
            outputs = self.ln(outputs)
        return outputs

        # feedforward

    def ff(self, inputs, num_units, scope="positionwise_feedforward"):

        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            # Inner layer
            outputs = tf.layers.dense(inputs, num_units[0], activation=tf.nn.relu)

            # Outer layer
            outputs = tf.layers.dense(outputs, num_units[1])

            # Residual connection
            outputs += inputs

            # Normalize
            outputs = self.ln(outputs)

        return outputs

    def TGRU_visit_cell(self, prev_h, concat_input):

        # visit_length x batch_size x input_dim+info_dim+info_dim+1
        batch_size = tf.shape(concat_input)[1]

        # 只获取第一个时间
        t = tf.slice(concat_input, [0, 0, 0], [1, batch_size, 1])
        # 只获取第一个info
        info = tf.slice(concat_input, [0, 0, 1], [1, batch_size, self.info_dim])

        # 循环创建局部变量
        for i in range(self.visit_length):
            locals()['x' + str(i)] = tf.slice(concat_input, [i, 0, 1 + self.info_dim],
                                              [1, batch_size, self.input_dim])

        # 获得visit层的输入
        visit_x = tf.zeros([batch_size, self.input_dim], dtype=tf.float32)
        for i in range(self.visit_length):
            visit_x += locals()['x' + str(i)]

        visit_x = visit_x / self.visit_length

        # 把输入和病人信息放在一个向量中
        visit_x = tf.concat([visit_x, info], 2)
        # 3维变2维 第一维是大小是1，没用
        visit_x = tf.reshape(visit_x, [tf.shape(visit_x)[1], tf.shape(visit_x)[2]])
        t = tf.reshape(t, [tf.shape(t)[1], tf.shape(t)[2]])

        # visit嵌入向量经过GRU输出
        ft = self.map_elapse_time(t, self.vhidden_dim)
        z = tf.sigmoid(tf.matmul(visit_x, self.Wz_v) + tf.matmul(prev_h, self.Uz_v) + self.bz_v)
        r = tf.sigmoid(tf.matmul(visit_x, self.Wr_v) + tf.matmul(prev_h, self.Ur_v) + self.br_v)
        d = tf.matmul(ft, self.Wd_v)
        h_ = tf.multiply(d, prev_h)
        h_canditate = tf.sigmoid(tf.matmul(visit_x, self.Wh_v) + tf.matmul(r * h_, self.Uh_v) + self.bh_v)
        current_h = h_ - z * h_ + z * h_canditate

        return current_h

    def encode(self, training=True):

        # batch_size x code_num x input_dim
        embeddings=self.get_input(self.one_hot_input)

        with tf.variable_scope("encoder", reuse=tf.AUTO_REUSE):

            # embedding normalization
            embeddings_norm = embeddings* (self.input_dim ** 0.5)
            enc = tf.nn.dropout(embeddings_norm, self.keep_prob)

            # Blocks n层attention
            for i in range(self.attention_blocks_num):
                with tf.variable_scope("num_blocks_{}".format(i), reuse=tf.AUTO_REUSE):
                    # self-attention
                    enc = self.multihead_attention(queries=enc,keys=enc,values=enc,num_heads=self.num_heads,keep_prob=self.keep_prob)
                    # feed forward
                    d_model = enc.get_shape().as_list()[-1]
                    enc = self.ff(enc, num_units=[self.f_dim, d_model])
        # enc (N, T1, d_model)
        memory = enc
        return memory


    def TGRU_visit(self):

        # batch_size x len x d_model)
        encode = self.encode()
        # visit_len x batch_size x d_model
        encode=tf.transpose(encode, [1,0,2])

        batch_size = tf.shape(encode)[1]
        scan_time = tf.transpose(self.time)  # scan_time [seq_length x batch_size]

        # make info [batch_size x seq_length x info_dim] --> [seq_length x batch_size x info_dim]
        scan_info = tf.transpose(self.info, [1, 0, 2])
        # make scan_time [seq_length x batch_size x 1]
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0], tf.shape(scan_time)[1], 1])

        # [seq_length x batch_size x input_dim+info_dim+1]
        concat_input = tf.concat([scan_time, scan_info, encode], 2)

        # 已经知道了visit的个数和长度，可以直接把concat_input拆分成visit个
        # [visit_num x visit_length x batch_size x input_dim+info_dim_1]
        initial_hidden = tf.zeros([batch_size, self.vhidden_dim], tf.float32)
        concat_input = tf.reshape(concat_input, [self.visit_num, self.visit_length, tf.shape(concat_input)[1],
                                                 tf.shape(concat_input)[2]])
        visit_h = tf.scan(self.TGRU_visit_cell, concat_input, initializer=initial_hidden, name='visit')

        return visit_h

    def get_cross_loss(self):
        output = self.TGRU_visit()
        batch_size = tf.shape(output)[1]
        final_output = tf.map_fn(self.get_output, output)
        fo = tf.zeros([batch_size, self.output_dim], dtype=tf.float32)
        for i in range(self.visit_num ):
            fo=fo+final_output[i]
        fo=fo/self.visit_num
        fo = tf.nn.dropout(fo, self.keep_prob)

        pre_output=self.get_final_output(fo)

        cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=self.labels, logits=pre_output))

        y_pred = tf.argmax(pre_output, 1)
        y = tf.argmax(self.labels, 1)

        return cross_entropy, y_pred, y, pre_output, self.labels


In [2]:
# batch生成器
# bacth中有cutting和padding，由于长度不同，因此开始学习的时候可以每个数据单独学习
# 生成的batch里的病人visit和code的个数一样
def batch_generator(outFile, visit_num, code_num, patient_num, batch_size):
    patient_code_file=open(outFile + '/patient_code' + '.seqs','rb')
    patient_code=pickle.load(patient_code_file)
    
    codespatientsinfo_file=open(outFile + '/codespatientsinfo' + '.seqs','rb')
    codespatientsinfo=pickle.load(codespatientsinfo_file)
    
    visit_delt_dates_file=open(outFile + '/visit_delt_dates' + '.seqs','rb')
    visit_delt_dates=pickle.load(visit_delt_dates_file)
    
    code_delt_dates_file=open(outFile + '/code_delt_dates' + '.seqs','rb')
    code_delt_dates=pickle.load(code_delt_dates_file)
    
    visits_num_file=open(outFile + '/visits_num' + '.seqs','rb')
    visits_num=pickle.load(visits_num_file)
    
    codes_num_file=open(outFile + '/codes_num' + '.seqs','rb')
    codes_num=pickle.load(codes_num_file)
    
    death_labels_file=open(outFile + '/death_labels' + '.seqs','rb')
    death_labels=pickle.load(death_labels_file)
    
    patient_code_file.close()
    codespatientsinfo_file.close()
    visit_delt_dates_file.close()
    code_delt_dates_file.close()
    visits_num_file.close()
    codes_num_file.close()

    batch_patient_code=[]
    batch_code_delt_dates=[]
    batch_codespatientsinfo=[]
    batch_visit_delt_dates=[]
    batch_labels=[]

    
    # padding and cutting
    for i in range(batch_size):
        j=random.randint(0, patient_num-1)
        if visits_num[j][0]== visit_num:
           # print 'pick',j
            code=[]
            code_date=[]
            code_info=[]
            for k in range(visit_num):
                if  codes_num[j][k]> code_num:
                    if k==0: 
                        code.extend(patient_code[j][0:code_num])
                        code_date.extend(code_delt_dates[j][0:code_num])
                        code_info.extend(codespatientsinfo[j][0:code_num])
                    else:
                        start=k*codes_num[j][k-1]
                        code.extend(patient_code[j][start :start+code_num])
                        code_date.extend(code_delt_dates[j][start:start+code_num])
                        code_info.extend(codespatientsinfo[j][start:start+code_num])
                elif codes_num[j][k] < code_num:
                    if k==0:
                        code.extend(patient_code[j][:codes_num[j][k]])
                        code.extend([0]*(code_num-codes_num[j][k]))
                        code_date.extend(code_delt_dates[j][:codes_num[j][k]])
                        code_date.extend([0]*(code_num-codes_num[j][k]))
                        code_info.extend(codespatientsinfo[j][:codes_num[j][k]])
                        code_info.extend([[0,0]]*(code_num-codes_num[j][k]))
                    else:
                        start2=0
                        for n in range(k):
                            start2+=codes_num[j][n]
                        code.extend(patient_code[j][start2: start2+codes_num[j][k]])
                        code.extend([0]*(code_num-codes_num[j][k]))
                        code_date.extend(code_delt_dates[j][start2:start2+codes_num[j][k]])
                        code_date.extend([0]*(code_num-codes_num[j][k]))
                        code_info.extend(codespatientsinfo[j][start2:start2+codes_num[j][k]])
                        code_info.extend([[0,0]]*(code_num-codes_num[j][k]))
                else:
                    code.extend(patient_code[j])
                    code_date.extend(code_delt_dates[j])
                    code_info.extend(codespatientsinfo[j])
            batch_patient_code.append(code)
            batch_code_delt_dates.append(code_date)  
            batch_codespatientsinfo.append(code_info)
            batch_labels.append(death_labels[j])
                    
        
        # 考虑随机生成的时候可能有的数据始终选不到，因此还要有一个顺序生成
    return batch_patient_code, batch_codespatientsinfo, batch_code_delt_dates,batch_labels

In [3]:
# 训练参数
learning_rate = 1e-3
iters = 20
dropout=1.0

# 网络参数
info_dim=2
icd9_num=259
one_hot_input_dim=icd9_num # 数据len(types)=259, one-hot的长度应该是259
input_dim = 200 
hidden_dim1 =150
output_dim1=150
hidden_dim2=150
output_dim2=150
vhidden_dim=100
voutput_dim=150
output_dim=200

# 生成batch数据参数
outFile='SEQ'
checkpoint_dir_pre='MODEL_pre'
result_dir='RESULT'
batch_size=4
visit_num=2 #visit_num=random.randint() 
code_num=10 #code_num=random.randint() # code_num=visit_length
patient_num=14 #病人的总个数



# 数据集获取
patient_code_file=open(outFile + '/patient_code' + '.seqs','rb')
patient_code=pickle.load(patient_code_file)

code_delt_dates_file=open(outFile + '/code_delt_dates' + '.seqs','rb')
code_delt_dates=pickle.load(code_delt_dates_file)

visit_delt_dates_file=open(outFile + '/visit_delt_dates' + '.seqs','rb')
visit_delt_dates=pickle.load(visit_delt_dates_file)

visits_num_file=open(outFile + '/visits_num' + '.seqs','rb')
visits_num=pickle.load(visits_num_file)

codes_num_file=open(outFile + '/codes_num' + '.seqs','rb')
codes_num=pickle.load(codes_num_file)

visitspatientsinfo_file=open(outFile + '/visitspatientsinfo' + '.seqs','rb')
visitspatientsinfo=pickle.load(visitspatientsinfo_file)

codespatientsinfo_file=open(outFile + '/codespatientsinfo' + '.seqs','rb')
codespatientsinfo=pickle.load(codespatientsinfo_file)

death_labels_file=open(outFile + '/death_labels' + '.seqs','rb')
death_labels=pickle.load(death_labels_file)


patient_code_file.close()
code_delt_dates_file.close()
visit_delt_dates_file.close()
visits_num_file.close()
codes_num_file.close()
visitspatientsinfo_file.close()
codespatientsinfo_file.close()
death_labels_file.close()

In [4]:
# 实例化网络
mtgrupredeath3 = MTGRU_predeath3(visit_num, code_num, one_hot_input_dim, input_dim, info_dim, output_dim1,
                 output_dim2, voutput_dim,output_dim, vhidden_dim, hidden_dim1, hidden_dim2,visits_num,codes_num,visit_delt_dates,visitspatientsinfo,patient_num,patient_code,code_delt_dates)




NameError: name 'MTGRU_predeath3' is not defined

In [ ]:
# 目标，使用交叉熵/ SME
cross_entropy, y_pred, y, logits, labels = mtgrupredeath3.get_cross_loss()
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess= tf.Session() 
sess.run(init)
Loss = np.zeros(iters)

In [10]:
 # 生成batch训练
for i in range(iters):
    Y_pred = []
    Y_true = []
    Labels = []
    Logits = []
    LOSS = 0 
    
    n=0
    for i in range(10): #学习完整个数据库需要的次数,要按照实际
        
        # 生成batch
        xindex, info, t,batch_labels = batch_generator(outFile, visit_num, code_num, patient_num,batch_size)
        if len(xindex)==0: continue
        _, c_train, y_pred_train, y_train, logits_train, labels_train = sess.run([optimizer, cross_entropy, y_pred, y, logits, labels ], feed_dict={mtgrupredeath3.inputindex: xindex, mtgrupredeath3.time: t, mtgrupredeath3.info:info,mtgrupredeath3.labels:batch_labels,mtgrupredeath3.keep_prob:dropout})
        LOSS += c_train
        
        if n > 0:
            Y_true = np.concatenate([Y_true, y_train], 0)
            Y_pred = np.concatenate([Y_pred, y_pred_train], 0)
            Labels = np.concatenate([Labels, labels_train], 0)
            Logits = np.concatenate([Logits, logits_train], 0)
        else:
            Y_true = y_train
            Y_pred = y_pred_train
            Labels = labels_train
            Logits = logits_train
        n+=1
    
    Loss[i] = LOSS / 10
    
    
    total_acc = accuracy_score(Y_true, Y_pred)
    total_auc = roc_auc_score(Labels, Logits, average='micro')
    total_auc_macro = roc_auc_score(Labels, Logits, average='macro')
    
    print('Loss, Accuracy, AUC, AUC Macro: %f %f %f %f' %(Loss[i],total_acc,total_auc,total_auc_macro))
    
    #print('Loss %f' %(Loss[i]))
    #print("Train Accuracy = {:.3f}".format(total_acc))
    #print("Train AUC = {:.3f}".format(total_auc))
    #print("Train AUC Macro = {:.3f}".format(total_auc_macro)) 

Loss, Accuracy, AUC, AUC Macro: 0.000002 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000002 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.000000 1.000000 1.000000
Loss, Accuracy, AUC, AUC Macro: 0.000001 1.00000

In [30]:
# 在整个数据集上的准确度
pres, codeAttention, visitAttention  = sess.run(mtgrupredeath3.get_all_Pre_sigmoid())

In [103]:
# 所有数据的准确度
def meanAcc(pres):
    test_pres=[]
    test_death_labels=[]
    for i in range(len(pres)):
        if pres[i][0][0]>pres[i][0][1]: test_pres.append(0)
        else: test_pres.append(1)
        test_death_labels.append(death_labels[i][1])

    format_pres=[]
    for pre in pres:
        format_pres.append(pre[0][1])

    all_acc = accuracy_score(test_death_labels, test_pres)
    all_auc = roc_auc_score(test_death_labels, format_pres, average='micro')
    all_auc_macro = roc_auc_score(test_death_labels, format_pres, average='macro')
    return all_acc,all_auc,all_auc_macro

In [139]:
# 每个病人的每个visit下最高贡献的code，死亡病人每个visit下最高贡献code
def PatientImpcode(codeAttention,patientAttention):
    all_codes_num_file= open(outFile + '/all_codes_num' + '.seqs', 'rb')
    all_codes_num=pickle.load(all_codes_num_file) 
    codes_num_file= open(outFile + '/codes_num' + '.seqs', 'rb')
    codes_num=pickle.load(codes_num_file) 
    newSeqs_file= open(outFile + '/newSeqs' + '.seqs', 'rb')
    newSeqs=pickle.load(newSeqs_file) 
    types_file= open(outFile + '/types2' + '.seqs', 'rb')
    types=pickle.load(types_file) 
    type_pids_file= open(outFile + '/type_pids2' + '.seqs', 'rb')
    type_pids=pickle.load(type_pids_file)
    death_labels_file=open(outFile + '/death_labels' + '.seqs','rb')
    death_labels=pickle.load(death_labels_file)
    visits_num_file=open(outFile + '/visits_num' + '.seqs','rb')
    visits_num=pickle.load(visits_num_file)
    newSeqs_file.close()
    types_file.close()
    type_pids_file.close()
    all_codes_num_file.close()
    death_labels_file.close()
    visits_num_file.close()

    #获取每个patient下的code的attention,每个visit的attention
    code_attentions_value=[]
    visit_attentions_value=[]
    for i in range(patient_num):
        code_attention_value=[]
        visit_attention_value=[]
        n=0
        for j in range(len(codes_num[i])):
            av=[]
            for k in range(codes_num[i][j]):
                av.append(np.linalg.norm(codeAttention[i][n]))
                n+=1
            code_attention_value.append(av)
        code_attentions_value.append(code_attention_value)
        for k in range(visits_num[i][0]):
            visit_attention_value.append(np.linalg.norm(visitAttention[i][k]))
        visit_attentions_value.append(visit_attention_value)

    # 对每个patient下的visit下code的attention大小进行排名,visit的attention大小排名
    code_attentions_rank=[]
    visit_attentions_rank=[]
    for i in range(patient_num):
        code_attention_rank=[]
        for j in range(len(codes_num[i])):
            rank=np.argsort(code_attentions_value[i][j])
            if j ==0 : code_attention_rank.append(rank[::-1])
            else:  code_attention_rank.append(rank[::-1]+codes_num[i][j-1])
        code_attentions_rank.append(code_attention_rank)
        rank2=np.argsort(visit_attentions_value[i])
        visit_attentions_rank.append(rank2[::-1])


    # 病人id对应的重要code的icd9编码
    PidImpcodeMap={}
    deathPidImpcodeMap={}
    PidMostImpcodeMap={}
    deathPidMostImpcodeMap={}
    longAffcode=[]
    for i in range(len(code_attentions_rank)):
        a=[]
        impCode=[]
        for j in range(len(code_attentions_rank[i])):
            a.append(types[patient_code[i][code_attentions_rank[i][j][0]]])
        impCode.append(a)
        PidImpcodeMap[type_pids[i]]=impCode
        PidMostImpcodeMap[type_pids[i]]=impCode[0][visit_attentions_rank[i][0]]
        # 死亡
        if death_labels[i][1]==1:
            deathPidImpcodeMap[type_pids[i]]=impCode
            deathPidMostImpcodeMap[type_pids[i]]=impCode[0][visit_attentions_rank[i][0]]
        # 长期影响的重要code（最重要的visit不是最后一个）
        if visit_attentions_rank[i][0] != (visits_num[i][0]-1) : longAffcode.append(impCode[0][visit_attentions_rank[i][0]])

    return PidImpcodeMap,deathPidImpcodeMap,PidMostImpcodeMap,deathPidMostImpcodeMap,longAffcode
    

In [140]:
all_acc,all_auc,all_auc_macro=meanAcc(pres)
PidImpcodeMap,deathPidImpcodeMap,PidMostImpcodeMap,deathPidMostImpcodeMap,longAffcode=PatientImpcode(codeAttention,patientAttention)
    
print('Accuracy, AUC, AUC Macro: %f %f %f' %(all_acc,all_auc,all_auc_macro))


Accuracy, AUC, AUC Macro: 0.785714 0.765306 0.765306


In [143]:
deathPidImpcodeMap

{10059: [['D_572.3', 'D_572.3']],
 10094: [['D_250.00', 'D_244.9']],
 10117: [['D_287.5', 'D_V42.81']],
 10124: [['D_250.00', 'D_599.0']],
 40310: [['D_707.03', 'D_227.3']],
 42135: [['D_303.93', 'D_585.9']],
 42346: [['D_397.0', 'D_458.9']]}

In [141]:
deathPidMostImpcodeMap

{10059: 'D_572.3',
 10094: 'D_244.9',
 10117: 'D_V42.81',
 10124: 'D_599.0',
 40310: 'D_227.3',
 42135: 'D_585.9',
 42346: 'D_458.9'}

肝脓肿与慢性肝病后遗症,门脉高压
甲状腺功能减退
骨髓移植
尿路感染
垂体和颅咽管良性肿瘤
慢性肾病
低血压

In [142]:
longAffcode

['D_284.1', 'D_038.3']

全血细胞减少症又名再生障碍性贫血(再障)，是骨髓造血功能衰竭所导致的一种全血减少综合征
厌氧菌败血症

In [6]:
xindex, info, t,batch_labels = batch_generator(outFile, visit_num, code_num, patient_num,batch_size)

In [7]:
xindex

[[98,
  3,
  0,
  99,
  100,
  49,
  8,
  101,
  102,
  53,
  108,
  3,
  109,
  110,
  102,
  25,
  111,
  112,
  20,
  11],
 [140,
  3,
  41,
  141,
  102,
  8,
  63,
  142,
  143,
  10,
  47,
  61,
  8,
  145,
  63,
  48,
  10,
  127,
  0,
  0],
 [38, 39, 25, 31, 40, 41, 42, 43, 35, 20, 44, 25, 45, 46, 11, 0, 0, 0, 0, 0],
 [98,
  3,
  0,
  99,
  100,
  49,
  8,
  101,
  102,
  53,
  108,
  3,
  109,
  110,
  102,
  25,
  111,
  112,
  20,
  11]]